In [1]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn import decomposition
from scipy import linalg
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline
np.set_printoptions(suppress=True)

In [3]:
categories = ['comp.os.ms-windows.misc', 'rec.autos', 'sci.electronics', 'sci.space']
remove = ('headers', 'footers', 'quotes')
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories, remove=remove)

In [4]:
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories, remove=remove)

In [5]:
newsgroups_train.filenames.shape, newsgroups_test.filenames.shape

((2369,), (1577,))

In [15]:
newsgroups_train.filenames.shape[0]

2369

In [28]:
random_index = np.random.randint(0, newsgroups_train.filenames.shape[0]-1, 3)
random_index

array([1701, 1844, 1911])

In [31]:
print("\n#----------#\n".join(np.array(newsgroups_train.data)[random_index]))

Hi: I'd like to know how much the foll. equipment will fetch in the used
equipment market (without manuals or other accessories):
	1. Tektronix 2465 scope
	2. Tektronix 2465A scope
	3. Tektronix 1240 logic analyser
#----------#
We have a LAN where we are doing development on product for multiple platforms. 
For the moment we are only working on MAC and DOS/Windows. The department has   
always used sneaker net to transport files to the MAC, since it requires a      
filter to strip out the <LF> characters.                                        

Until recently no one concidered using any version control to mediate, and as   
a result, the 5 programmers spent a great deal of time merging files together   
at the end of each week so that a new system could be build. We are now trying  
to streamline this process, but are hampered by the lack of software that will  
allow us to share files across PC and MAC platforms.                            

I understand that PVCS used to do this, b

In [32]:
np.array(newsgroups_train.target_names)[newsgroups_train.target[random_index]]

array(['sci.electronics', 'comp.os.ms-windows.misc', 'sci.space'],
      dtype='<U23')

In [33]:
newsgroups_train.target[:10]

array([0, 0, 0, 2, 0, 2, 2, 1, 1, 0], dtype=int64)

In [34]:
num_topics, num_top_words = 6, 8

# Stop words

In [35]:
from sklearn.feature_extraction import stop_words
sorted(list(stop_words.ENGLISH_STOP_WORDS))[:20]

['a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst']